In [1]:
import numpy as np 
import pandas as pd
import sqlite3
import csv
from figure_labeler import *

from IPython.display import HTML
HTML('''
<script
    src="https://cdnjs.cloudflare.com/ajax/libs/jquery/2.0.3/jquery.min.js ">
</script>
<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.jp-CodeCell > div.jp-Cell-inputWrapper').hide();
 } else {
$('div.jp-CodeCell > div.jp-Cell-inputWrapper').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit"
    value="Click here to toggle on/off the raw code."></form>
''')

In [2]:
fl = FigureLabeler();
pd.options.mode.chained_assignment = None

<h1 style="text-align:center;">Using Pandas and SQL for EDA</h1>
<hr>

<a name="top"></a>
#### Table of Contents:

[ref0]: #exec_summary
- [Executive Summary][ref0]

[ref2]: #motiv
- [Motivation][ref2]

[ref3]: #dat_sor
- [Data Source][ref3]

[ref4]: #dat_prep
- [Importing, Preprocessing, and EDA][ref4]

[ref6]: #res_dis
- [Conclusion][ref6]

***

<a name="exec_summary"></a>
## Executive Summary
***

The objective is to demonstrate the use of Pandas and SQL for exploratory data analysis (EDA) through two separate sections in the Jupyter notebook. One section will focus on Pandas while the other on SQL, showcasing their respective strengths in data manipulation and querying. The analysis will highlight how these tools can be seamlessly integrated to efficiently explore and gain insights from datasets, thereby providing a comprehensive understanding of the data's characteristics and patterns.

[ref]: #top
[Back to Table of Contents][ref]

<a name="motiv"></a>
## Motivation
***

Motivated by the need for comprehensive exploratory data analysis (EDA) workflows, we aim to showcase the powerful synergy and similarities between two widely-used tools: Pandas and SQL. While Pandas excels in data manipulation and analysis within Python environments, SQL offers robust querying capabilities ideal for handling large datasets efficiently.

[ref]: #top
[Back to Table of Contents][ref]

<a name="dat_sor"></a>
## Data Source
***

The dataset was taken from this link
[Student Study Performance Dataset](https://www.kaggle.com/datasets/bhavikjikadara/student-study-performance)


[ref]: #top
[Back to Table of Contents][ref]

<a name="dat_prep"></a>
## Importing, Preprocessing and EDA
***

In our SQL-based exploratory data analysis (EDA), we first imported the dataset into a SQL database management system using commands like CREATE TABLE and LOAD DATA, ensuring efficient storage and organization for analysis. Subsequently, we performed preprocessing tasks through SQL queries, including data cleaning, handling missing values, and ensuring data consistency. Leveraging SQL's querying capabilities, we conducted exploratory analysis by crafting queries involving aggregation functions like COUNT, SUM, and AVG to generate summary statistics for numerical attributes, while GROUP BY and ORDER BY clauses facilitated exploration of categorical variables. Throughout this process, we prioritized query optimization for performance and scalability, ensuring timely execution even with large datasets, and laying a robust foundation for subsequent analytical endeavors.

In our Pandas exploratory data analysis (EDA), we harnessed the power of Pandas to import, preprocess, and gain insights from the dataset. Employing Pandas' read_csv() function, we loaded the data into a DataFrame, then utilized methods like .head() to preview the initial rows, .dtypes to inspect data types, and .describe() to generate summary statistics for numerical columns. By employing .value_counts(), we explored categorical variable distributions and used .isnull().sum() to identify missing values, crucial for data integrity. Additionally, .nunique() aided in understanding unique value counts per column. This meticulous process laid a robust foundation for extracting meaningful insights, guiding subsequent analytical steps effectively.

[ref]: #top
[Back to Table of Contents][ref]

## SQL

In [3]:
conn = sqlite3.connect('performance_data.db')
cursor = conn.cursor()

create_table_query = '''
CREATE TABLE IF NOT EXISTS performance (
    gender TEXT,
    race_ethnicity TEXT,
    parental_level_of_education TEXT,
    lunch TEXT,
    test_preparation_course TEXT,
    math_score INTEGER,
    reading_score INTEGER,
    writing_score INTEGER
);
'''
cursor.execute(create_table_query)

with open('study_performance.csv', 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip header row
    for row in csv_reader:
        cursor.execute("INSERT INTO performance VALUES (?, ?, ?, ?, ?, ?, ?, ?)", row)

conn.commit()
conn.close()

print("Data imported successfully into SQLite database.")

Data imported successfully into SQLite database.


In [4]:
conn = sqlite3.connect('performance_data.db')
cursor_head = conn.cursor()

cursor_head.execute("SELECT * FROM performance LIMIT 5")

rows = cursor_head.fetchall()
for row in rows:
    print(row)

conn.close()


('female', 'group B', "bachelor's degree", 'standard', 'none', 72, 72, 74)
('female', 'group C', 'some college', 'standard', 'completed', 69, 90, 88)
('female', 'group B', "master's degree", 'standard', 'none', 90, 95, 93)
('male', 'group A', "associate's degree", 'free/reduced', 'none', 47, 57, 44)
('male', 'group C', 'some college', 'standard', 'none', 76, 78, 75)


First 5 rows were printed from the database.

In [5]:
conn = sqlite3.connect('performance_data.db')
cursor_dtypes = conn.cursor()

data_types_query = """
PRAGMA table_info(performance);
"""

cursor_dtypes.execute(data_types_query)

data_types = cursor_dtypes.fetchall()

columns = ['cid', 'name', 'type', 'notnull', 'dflt_value', 'pk']
data_types_df = pd.DataFrame(data_types, columns=columns)
print(data_types_df)

conn.close()

   cid                         name     type  notnull dflt_value  pk
0    0                       gender     TEXT        0       None   0
1    1               race_ethnicity     TEXT        0       None   0
2    2  parental_level_of_education     TEXT        0       None   0
3    3                        lunch     TEXT        0       None   0
4    4      test_preparation_course     TEXT        0       None   0
5    5                   math_score  INTEGER        0       None   0
6    6                reading_score  INTEGER        0       None   0
7    7                writing_score  INTEGER        0       None   0


Checking for Each type of data within the database

In [6]:
conn = sqlite3.connect('performance_data.db')
cursor_sumstats = conn.cursor()

cursor_sumstats.execute("""
SELECT * FROM performance;
""")
summary_stats = cursor_sumstats.fetchall()
columns = [description[0] for description in cursor_sumstats.description]
df = pd.DataFrame(summary_stats, columns=columns)
print(df.describe())

conn.close()

       math_score  reading_score  writing_score
count  1000.00000    1000.000000    1000.000000
mean     66.08900      69.169000      68.054000
std      15.16308      14.600192      15.195657
min       0.00000      17.000000      10.000000
25%      57.00000      59.000000      57.750000
50%      66.00000      70.000000      69.000000
75%      77.00000      79.000000      79.000000
max     100.00000     100.000000     100.000000


Describing the Data using df.describe

In [7]:
conn = sqlite3.connect('performance_data.db')
cursor_gender_counts = conn.cursor()

gender_counts_query = """
SELECT gender, COUNT(*) AS count
FROM performance
GROUP BY gender;
"""

cursor_gender_counts.execute(gender_counts_query)

gender_counts = cursor_gender_counts.fetchall()

for gender, count in gender_counts:
    print(f"{gender}: {count}")

conn.close()

female: 518
male: 482


Counts of Gender Distribution.

In [8]:
conn = sqlite3.connect('performance_data.db')
cursor_race_counts = conn.cursor()

race_counts_query = """
SELECT race_ethnicity, COUNT(*) AS count
FROM performance
GROUP BY race_ethnicity;
"""

cursor_race_counts.execute(race_counts_query)

race_counts = cursor_race_counts.fetchall()

for race, count in race_counts:
    print(f"{race}: {count}")

conn.close()

group A: 89
group B: 190
group C: 319
group D: 262
group E: 140


Count of Ethnicity Groups of Students

In [9]:
conn = sqlite3.connect('performance_data.db')
cursor_null = conn.cursor()

table_schema_query = """
PRAGMA table_info(performance);
"""
cursor_null.execute(table_schema_query)

table_schema = cursor_null.fetchall()

missing_values_query = """
SELECT {}
FROM performance;
"""

column_exprs = []
for col in table_schema:
    col_name = col[1]  
    col_expr = f"SUM(CASE WHEN {col_name} IS NULL THEN 1 ELSE 0 END) AS missing_{col_name}"
    column_exprs.append(col_expr)

column_exprs_str = ', '.join(column_exprs)

missing_values_query = missing_values_query.format(column_exprs_str)

cursor_null.execute(missing_values_query)

missing_values = cursor_null.fetchone()

print(missing_values)

conn.close()

(0, 0, 0, 0, 0, 0, 0, 0)


No NULL Values

In [10]:
conn = sqlite3.connect('performance_data.db')
cursor_nunique = conn.cursor()

column_names_query = """
PRAGMA table_info(performance);
"""

cursor_nunique.execute(column_names_query)

column_names = [row[1] for row in cursor_nunique.fetchall()]

nunique_query = f"""
SELECT {', '.join([f'COUNT(DISTINCT {col}) AS unique_{col}' for col in column_names])}
FROM performance;
"""

cursor_nunique.execute(nunique_query)

nunique_results = cursor_nunique.fetchone()

print("Number of unique values in each column:")
for column, unique_count in zip(column_names, nunique_results):
    print(f"{column}: {unique_count}")

conn.close()

Number of unique values in each column:
gender: 2
race_ethnicity: 5
parental_level_of_education: 6
lunch: 2
test_preparation_course: 2
math_score: 81
reading_score: 72
writing_score: 77


Various Unique Values of our Dataset.

## PANDAS

In [6]:
perf = pd.read_csv('study_performance.csv')

In [7]:
fl.table_caption("Head of Study Performance of Students",
                 "The first 5 rows of the Data on Study Performance of Students")
perf.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [8]:
perf.dtypes

gender                         object
race_ethnicity                 object
parental_level_of_education    object
lunch                          object
test_preparation_course        object
math_score                      int64
reading_score                   int64
writing_score                   int64
dtype: object

The different types Data in our dataset.

In [9]:
fl.table_caption("Describe of Study Performance",
                 "Description of different variables using .describe on the data")
perf.describe()

,math_score,reading_score,writing_score
count,1000.00000,1000.000000,1000.000000
mean,66.08900,69.169000,68.054000
std,15.16308,14.600192,15.195657
min,0.00000,17.000000,10.000000
25%,57.00000,59.000000,57.750000
50%,66.00000,70.000000,69.000000
75%,77.00000,79.000000,79.000000
max,100.00000,100.000000,100.000000


In [10]:
perf['gender'].value_counts()

gender
female    518
male      482
Name: count, dtype: int64

Count of Gender Distribution

In [11]:
perf['race_ethnicity'].value_counts()

race_ethnicity
group C    319
group D    262
group B    190
group E    140
group A     89
Name: count, dtype: int64

Count of Ethnicity Groups of Students

In [12]:
perf.isnull().sum()

gender                         0
race_ethnicity                 0
parental_level_of_education    0
lunch                          0
test_preparation_course        0
math_score                     0
reading_score                  0
writing_score                  0
dtype: int64

No NULL Values

In [13]:
perf.nunique()

gender                          2
race_ethnicity                  5
parental_level_of_education     6
lunch                           2
test_preparation_course         2
math_score                     81
reading_score                  72
writing_score                  77
dtype: int64

Various Unique Values of our Dataset.

<a name="res_dis"></a>
## Conclusion
***

In conclusion, our exploratory data analysis (EDA) showcased the efficacy and versatility of leveraging both SQL and Pandas for data import, preprocessing, and analysis tasks. Through SQL's querying capabilities, we imported the dataset into a database and executed queries to preprocess and explore the data, obtaining valuable insights into its characteristics and distributions. This process, coupled with Pandas' functionalities, allowed for seamless integration and comprehensive understanding of the dataset. By utilizing aggregation functions, grouping operations, and Pandas' rich functionalities, we ensured data integrity and prepared a solid groundwork for further analysis. Moreover, our emphasis on query optimization and meticulous examination of data types and missing values underscore the importance of efficiency and data integrity in handling diverse datasets. Overall, this combined approach highlights the power of both SQL and Pandas as fundamental tools for data exploration, empowering practitioners to derive actionable insights and make informed decisions across diverse domains.



[ref]: #top
[Back to Table of Contents][ref]